In [1]:
""" This demo solves the Stokes equations, using quadratic elements for
the velocity and first degree elements for the pressure (Taylor-Hood elements)."""
# Credits: https://github.com/pf4d/fenics_scripts/blob/master/cbc_block/stokes.py
from dolfin import *
from time import time
from vtkplotter.dolfin import datadir

# Load mesh and subdomains
mesh = Mesh(datadir+"dolfin_fine.xml")
sub_domains = MeshFunction("size_t", mesh, datadir+"dolfin_fine_subdomains.xml.gz")

# Define function spaces
P2 = VectorElement("Lagrange", mesh.ufl_cell(), 2)
P1 = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
TH = P2 * P1
W = FunctionSpace(mesh, TH)

# No-slip boundary condition for velocity
noslip = Constant((0, 0))
bc0 = DirichletBC(W.sub(0), noslip, sub_domains, 0)

# Inflow boundary condition for velocity
inflow = Expression(("-sin(x[1]*pi)", "0.0"), degree=2)
bc1 = DirichletBC(W.sub(0), inflow, sub_domains, 1)
bcs = [bc0, bc1]

# Define variational problem
(u, p) = TrialFunctions(W)
(v, q) = TestFunctions(W)
f = Constant((0, 0))
a = (inner(grad(u), grad(v)) - div(v)*p + q*div(u))*dx
L = inner(f, v)*dx
w = Function(W)

solve(a == L, w, bcs) #solver_parameters={'linear_solver':'mumps'}
(u, p) = w.split() # Split the mixed solution using a shallow copy

######################################################## vtkplotter:
from vtkplotter.dolfin import plot, embedWindow

embedWindow('itkwidgets') # backends are: itkwidgets, k3d or False

plot(u, mode='mesh and arrows', warpZfactor=-0.05, scale=0.02)

Viewer(cmap='jet', geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_point…